In [16]:
import sagemaker
from sagemaker import get_execution_role

role = get_execution_role()
sess = sagemaker.Session()

default_bucket = sess.default_bucket() # or use your own custom bucket name
account = sess.account_id()
base_job_prefix = 'BirdEnd2End'
region = sagemaker.Session().boto_region_name

In [17]:
container_name = "sagemaker-tf-container"
# !cd docker && sm-docker build . --file Dockerfile --repository $container_name:2.0
    
ecr_image = "{}.dkr.ecr.{}.amazonaws.com/{}:2.0".format(account, region, container_name)
ecr_image

'987720697751.dkr.ecr.us-west-2.amazonaws.com/sagemaker-tf-container:2.0'

In [20]:
import boto3
from sagemaker.processing import ScriptProcessor, ProcessingInput, ProcessingOutput, Processor
from sagemaker import get_execution_role

s3_data = f's3://{default_bucket}/{base_job_prefix}/scriptprocessor/output/preprocess'
s3_model = f"s3://{default_bucket}/{base_job_prefix}/BirdEnd2End-debugger-2022-03-17-04-33-10-008/output"

role = get_execution_role()

image_uri = ecr_image

script_processor = ScriptProcessor(
                base_job_name = f"{base_job_prefix}-evaluation",
                command=['python3'],
                image_uri=image_uri,
                role=role,
                instance_count=1,
                instance_type='ml.m5.xlarge')

In [21]:
script_processor.run(
                code='pipeline/evaluation.py',
                arguments=["--model-file", "model.tar.gz",
                          "--classes-file", "classes.json"],
                inputs=[ProcessingInput(source=f"{s3_data}/test", 
                                        destination="/opt/ml/processing/input/test"),
                        ProcessingInput(source=f"{s3_data}/classes", 
                                        destination="/opt/ml/processing/input/classes"),
                        ProcessingInput(source=s3_model, 
                                        destination="/opt/ml/processing/model"),
                       ],
                outputs=[
                    ProcessingOutput(output_name="evaluation", source="/opt/ml/processing/evaluation",
                                    destination=f"s3://{default_bucket}/{base_job_prefix}/scriptprocessor/output/evaluation")
                ]
            )


Job Name:  BirdEnd2End-evaluation-2022-03-17-04-56-00-703
Inputs:  [{'InputName': 'input-1', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-west-2-987720697751/BirdEnd2End/scriptprocessor/output/preprocess/test', 'LocalPath': '/opt/ml/processing/input/test', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'input-2', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-west-2-987720697751/BirdEnd2End/scriptprocessor/output/preprocess/classes', 'LocalPath': '/opt/ml/processing/input/classes', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'input-3', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-us-west-2-987720697751/BirdEnd2End/BirdEnd2End-debugger-2022-03-17-04-33-10-008/output', 'LocalPath': '/opt/ml/processing/model', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3